In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [11]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [10]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")


In [26]:
driver = webdriver.Chrome(options=options)
driver.get(standing_url)
time.sleep(5)

soup = BeautifulSoup(driver.page_source, "html.parser")
standings_table = soup.select_one("table.stats_table")

if standings_table is None:
    print("❌ Could not find the standings table.")
    driver.quit()
else:
    links = [a.get("href") for a in standings_table.find_all("a") if "/squads/" in a.get("href", "")]
    base_url = "https://fbref.com"
    team_urls = [base_url + link for link in links]

    print(f"✅ Found {len(team_urls)} teams")
    print(team_urls[:3])

    # --- Lấy dữ liệu đội đầu tiên ---
    team_url = team_urls[0]
    print(f"\n🔗 Getting team data from: {team_url}")
    driver.get(team_url)
    time.sleep(5)

    # --- Lấy bảng Scores & Fixtures ---
    team_soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")
    matches = tables[0]
    print("\n✅ Sample of matches table:")
    print(matches.head())

    # --- Tìm link đến bảng Shooting ---
    links = [a.get("href") for a in team_soup.find_all("a") if a.get("href")]
    shooting_links = [l for l in links if "all_comps/shooting/" in l]

    if shooting_links:
        shooting_url = f"https://fbref.com{shooting_links[0]}"
        print(f"\n📊 Shooting stats URL: {shooting_url}")

        driver.get(shooting_url)
        time.sleep(5)

        shooting_soup = BeautifulSoup(driver.page_source, "html.parser")
        shooting_tables = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")
        shooting = shooting_tables[0]
        print("\n✅ Sample of shooting table:")
        print(shooting.head())
    else:
        print("⚠️ No shooting table found for this team.")

driver.quit()

✅ Found 20 teams
['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats', 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats', 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats']

🔗 Getting team data from: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3548\2686024295.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")



✅ Sample of matches table:
         Date           Time            Comp         Round  Day Venue Result  \
0  2025-08-17  16:30 (22:30)  Premier League   Matchweek 1  Sun  Away      W   
1  2025-08-23  17:30 (23:30)  Premier League   Matchweek 2  Sat  Home      W   
2  2025-08-31  16:30 (22:30)  Premier League   Matchweek 3  Sun  Away      L   
3  2025-09-13  12:30 (18:30)  Premier League   Matchweek 4  Sat  Home      W   
4  2025-09-16  18:45 (23:45)    Champions Lg  League phase  Tue  Away      W   

  GF GA          Opponent   xG  xGA Poss Attendance            Captain  \
0  1  0    Manchester Utd  1.3  1.5   39      73475    Martin Ødegaard   
1  5  0      Leeds United  2.7  0.2   67      60110    Martin Ødegaard   
2  0  1         Liverpool  0.5  0.5   47      60455  Gabriel Magalhães   
3  3  0   Nott'ham Forest  1.8  0.2   54      60167    Martin Ødegaard   
4  2  0  es Athletic Club  1.3  0.3   61      51059  Gabriel Magalhães   

  Formation Opp Formation         Referee  Mat

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3548\2686024295.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting_tables = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")



✅ Sample of shooting table:
  For Arsenal                                                                 \
         Date           Time            Comp         Round  Day Venue Result   
0  2025-08-17  16:30 (22:30)  Premier League   Matchweek 1  Sun  Away      W   
1  2025-08-23  17:30 (23:30)  Premier League   Matchweek 2  Sat  Home      W   
2  2025-08-31  16:30 (22:30)  Premier League   Matchweek 3  Sun  Away      L   
3  2025-09-13  12:30 (18:30)  Premier League   Matchweek 4  Sat  Home      W   
4  2025-09-16  18:45 (23:45)    Champions Lg  League phase  Tue  Away      W   

                               ... Standard               Expected       \
    GF   GA          Opponent  ...     Dist   FK PK PKatt       xG npxG   
0  1.0  0.0    Manchester Utd  ...     18.6  1.0  0     0      1.3  1.3   
1  5.0  0.0      Leeds United  ...     15.2  0.0  1     1      2.7  1.9   
2  0.0  1.0         Liverpool  ...     17.5  1.0  0     0      0.5  0.5   
3  3.0  0.0   Nott'ham Forest  ... 

In [27]:
shooting.columns = shooting.columns.droplevel()    

In [28]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2025-08-17,16:30 (22:30),Premier League,Matchweek 1,Sun,Away,W,1.0,0.0,Manchester Utd,...,18.6,1.0,0,0,1.3,1.3,0.15,-0.3,-0.3,Match Report
1,2025-08-23,17:30 (23:30),Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Leeds United,...,15.2,0.0,1,1,2.7,1.9,0.12,2.3,2.1,Match Report
2,2025-08-31,16:30 (22:30),Premier League,Matchweek 3,Sun,Away,L,0.0,1.0,Liverpool,...,17.5,1.0,0,0,0.5,0.5,0.04,-0.5,-0.5,Match Report
3,2025-09-13,12:30 (18:30),Premier League,Matchweek 4,Sat,Home,W,3.0,0.0,Nott'ham Forest,...,13.5,0.0,0,0,1.8,1.8,0.11,1.2,1.2,Match Report
4,2025-09-16,18:45 (23:45),Champions Lg,League phase,Tue,Away,W,2.0,0.0,es Athletic Club,...,13.5,0.0,0,0,1.3,1.3,0.12,0.7,0.7,Match Report


In [32]:
team_data = matches.merge(shooting[['Date','Sh','SoT','Dist','FK','PK','PKatt']], on='Date', how='left')

In [38]:
team_data.shape

(52, 26)

In [41]:
team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

'Arsenal'

In [12]:
years = list(range(2025, 2020, -1))

In [13]:
all_matches = []

In [14]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
base_url = "https://fbref.com"

In [15]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")

In [16]:
driver = webdriver.Chrome(options=options)

try:
    for year in years:
        print(f"\n📅 Getting data for season {year}...")
        driver.get(standing_url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        standings_table = soup.select_one("table.stats_table")

        if standings_table is None:
            print("❌ Could not find the standings table.")
            break

        team_links = [a.get("href") for a in standings_table.find_all("a") if "/squads/" in a.get("href", "")]
        team_urls = [base_url + link for link in team_links]
        print(f"✅ Found {len(team_urls)} teams")
        
        for i, team_url in enumerate(team_urls, start=1):
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            print(f"\n[{i}/{len(team_urls)}] Getting data for: {team_name}")
            
            try:
                driver.get(team_url)
                time.sleep(5)
                team_soup = BeautifulSoup(driver.page_source, "html.parser")
                
                # Get matches
                matches = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")[0]
                
                # Find shooting links
                links = [a.get("href") for a in team_soup.find_all("a") if a.get("href")]
                shooting_links = [l for l in links if "all_comps/shooting/" in l]
                
                if not shooting_links:
                    print("⚠️ No shooting record found for this team.")
                    continue
                
                # Get shooting data
                shooting_url = f"{base_url}{shooting_links[0]}"
                driver.get(shooting_url)
                time.sleep(5)
                shooting_soup = BeautifulSoup(driver.page_source, "html.parser")
                
                shooting = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")[0]
                shooting.columns = shooting.columns.droplevel()
                
                # Merge data
                team_data = matches.merge(shooting[['Date','Sh','SoT','Dist','FK','PK','PKatt']], on='Date', how='left')
                team_data = team_data[team_data["Comp"] == "Premier League"].copy()
                team_data["Season"] = year
                team_data["Team"] = team_name
                all_matches.append(team_data)
                print(f"✅ Done: {team_name} ({len(team_data)} matches)")
                
            except Exception as e:
                print(f"❌ Error processing {team_name}: {e}")
                # Recreate driver if session is lost
                try:
                    driver.quit()
                except:
                    pass
                driver = webdriver.Chrome(options=options)
                continue
            
            time.sleep(3)  # Rate limiting

        prev_link = soup.select_one("a.prev")
        if not prev_link:
            print("🏁 No previous season link found, ending.")
            break
        standing_url = base_url + prev_link.get("href")

finally:
    driver.quit()


📅 Getting data for season 2025...
✅ Found 20 teams

[1/20] Getting data for: Arsenal


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")[0]


✅ Done: Arsenal (38 matches)

[2/20] Getting data for: Liverpool


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")[0]


✅ Done: Liverpool (38 matches)

[3/20] Getting data for: Tottenham Hotspur


...

✅ Done: Burnley (38 matches)

[19/20] Getting data for: Watford


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")[0]


✅ Done: Watford (38 matches)

[20/20] Getting data for: Norwich City


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(str(team_soup), match="Scores & Fixtures", flavor="lxml")[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15860\273367810.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(str(shooting_soup), match="Shooting", flavor="lxml")[0]


✅ Done: Norwich City (38 matches)


In [17]:
final_df = pd.concat(all_matches, ignore_index=True)
print("\n✅ All data collected!")
print(final_df.head())

# --- Lưu file ---
final_df.to_csv("premier_league_multi_season.csv", index=False)
print("\n💾 Saved to premier_league_multi_season.csv")


✅ All data collected!
         Date           Time            Comp        Round  Day Venue Result  \
0  2025-08-17  16:30 (22:30)  Premier League  Matchweek 1  Sun  Away      W   
1  2025-08-23  17:30 (23:30)  Premier League  Matchweek 2  Sat  Home      W   
2  2025-08-31  16:30 (22:30)  Premier League  Matchweek 3  Sun  Away      L   
3  2025-09-13  12:30 (18:30)  Premier League  Matchweek 4  Sat  Home      W   
4  2025-09-21  16:30 (22:30)  Premier League  Matchweek 5  Sun  Home      D   

  GF GA         Opponent  ...  Match Report Notes    Sh  SoT  Dist   FK   PK  \
0  1  0   Manchester Utd  ...  Match Report   NaN   9.0  3.0  18.6  1.0  0.0   
1  5  0     Leeds United  ...  Match Report   NaN  17.0  4.0  15.2  0.0  1.0   
2  0  1        Liverpool  ...  Match Report   NaN  11.0  1.0  17.5  1.0  0.0   
3  3  0  Nott'ham Forest  ...  Match Report   NaN  16.0  5.0  13.5  0.0  0.0   
4  1  1  Manchester City  ...  Match Report   NaN  12.0  3.0  14.1  0.0  0.0   

  PKatt Season     Te